In [6]:
void p_vs_beta(const std::vector<std::string>& filepaths) {
    TFile *outfile = new TFile("momentum_vs_beta.root", "RECREATE");
    TH2F *pVSbeta = new TH2F("momentum_vs_beta", "P vs beta; p [GeV/c]; beta;", 200, 0.36, 3.2, 200, 0.5, 1);

    int total_entries = 0;
    int passed_entries = 0;

    for (const auto& filepath : filepaths) {
        TFile *inputfile = TFile::Open(filepath.c_str());
        if (!inputfile || inputfile->IsZombie()) {
            std::cerr << "Error: Can't open input file " << filepath << std::endl;
            continue;
        }

        TList *keys = inputfile->GetListOfKeys();
        if (!keys) {
            std::cerr << "Error: Failed to get list of keys in file " << filepath << std::endl;
            inputfile->Close();
            continue;
        }

        TIter next(keys);
        TKey *key;
        while ((key = (TKey*)next())) {
            if (strcmp(key->GetClassName(), "TDirectoryFile") == 0) {
                TDirectoryFile *dir = (TDirectoryFile*)key->ReadObj();
                TTree *treeReducedTrack = dir->Get<TTree>("O2reducedtrack");
                TTree *treeRtBarrelpid = dir->Get<TTree>("O2rtbarrelpid");
                TTree *treeRtBarrel = dir->Get<TTree>("O2rtbarrel");

                if (!treeReducedTrack || !treeRtBarrelpid || !treeRtBarrel) {
                    std::cerr << "Error: Missing required trees in file " << filepath << std::endl;
                    delete dir;
                    continue;
                }

                Long64_t nEntries = treeReducedTrack->GetEntries();
                if (treeRtBarrelpid->GetEntries() != nEntries || treeRtBarrel->GetEntries() != nEntries) {
                    std::cerr << "Error: Trees in " << filepath << " have mismatched entries!" << std::endl;
                    delete dir;
                    continue;
                }

                float pT, pseudorap, beta;
                UChar_t IsFindable;
                treeReducedTrack->SetBranchAddress("fPt", &pT);
                treeReducedTrack->SetBranchAddress("fEta", &pseudorap);
                treeRtBarrelpid->SetBranchAddress("fBeta", &beta);
                treeRtBarrel->SetBranchAddress("fTPCNClsFindable", &IsFindable);

                for (Long64_t iEntry = 0; iEntry < nEntries; ++iEntry) {
                    treeReducedTrack->GetEntry(iEntry);
                    treeRtBarrelpid->GetEntry(iEntry);
                    treeRtBarrel->GetEntry(iEntry);
                    
                    
                    if (IsFindable >= 100) {
                    pVSbeta->Fill(pT * std::cosh(pseudorap), beta);
                     
                    passed_entries++; }
                }
                delete dir;
            }
        }
        inputfile->Close();
        delete inputfile;
    }

    outfile->Write();
    outfile->Close();
    delete outfile;

    std::cout << "Analysis complete. Results saved to momentum_vs_beta.root" << std::endl;
}
std::vector<std::string> filepaths = {
        "AO2D.root",
        "AO2D_798.root",
        "AO2D_797.root",
        "AO2D_796.root",
        "AO2D_795.root",
        "AO2D_794.root",
        "AO2D_793.root",
        "AO2D_792.root",
        "AO2D_791.root",
        "AO2D_790.root"
    };

p_vs_beta(filepaths);
 

Analysis complete. Results saved to momentum_vs_beta.root


In [7]:
void PlotMomentumVSbeta() {

    TFile *file = TFile::Open("momentum_vs_beta.root");

    if (!file || file -> IsZombie()) {
        std::cout << "Error: Couldnt open momentum_vs_beta.root" << std::endl;
        return;
    }
    TH2D *p_vs_beta = (TH2D*)file->Get("momentum_vs_beta");
    if (!p_vs_beta) {
        std::cout << "no histogram" << std::endl;
        return;
    }

    auto pvsbeta = new TCanvas("c3", "2D Histogram", 50,150, 800, 600);
    gStyle->SetPalette(kDarkBodyRadiator);
    gStyle->SetOptStat(0); 
    gPad->SetLogz();  

    
    p_vs_beta->Draw();
    pvsbeta->Update();
    pvsbeta->SaveAs("momentumVSbeta.png");
}

PlotMomentumVSbeta();

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Info in <TCanvas::Print>: png file momentumVSbeta.png has been created
